In [1]:
import csv
import networkx as nx
import numpy as np
from random import randint
from random import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, accuracy_score
from random import choice
from gensim.models import Word2Vec
import keras

2023-03-12 13:59:18.977272: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from scipy.sparse import identity, diags

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
np.ones(4).shape

(4,)

In [15]:
def normalize_adjacency(A):
    n = A.shape[0]
    A = A + identity(n)
    degs = A.dot(np.ones(n))
    print('degs shape:', degs.shape)
    inv_degs = np.power(degs, -1)
    print('inv_degs', inv_degs.shape)
    D_inv = diags(inv_degs)
    print('D_inv shape:', D_inv.shape)
    A_hat = D_inv.dot(A)
    print('A_hat shape:', A_hat.shape)
    return A_hat

In [4]:
def new_normalize_adjacency(A):
    n = A.shape[0]
    A = A + identity(n)
    #degs = A.dot(np.ones(n))
    #inv_degs = np.power(degs, -1)
    #D_inv = diags(inv_degs)
    #A_hat = D_inv.dot(A)
    return A

In [12]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    print(type(sparse_mx))
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

# Is it equivalent to Word2vec for dimensionality reduction ? --> NO !

In [22]:
class GNN(nn.Module):
    def __init__(self, n_feat, n_hidden, n_class, dropout):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(n_feat, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.fc4 = nn.Linear(n_hidden, n_class)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x_in, adj, pairs):
        
        h1 = self.fc1(x_in)
        z1 = self.relu(torch.mm(adj, h1))
        z1 = self.dropout(z1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.mm(adj, h2))
        
        x = z2[pairs[0,:],:] - z2[pairs[1,:],:]
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return F.log_softmax(x, dim=1)

In [11]:

G = nx.read_edgelist('../input_data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
edges = list(G.edges())

val_edges = list()
G_train = G

for edge in edges:
    if random() < 0.1:
        val_edges.append(edge)

# We remove the val edges from the graph G
for edge in val_edges:
    G_train.remove_edge(edge[0], edge[1])

n = G_train.number_of_nodes()
m = G_train.number_of_edges()
train_edges = list(G_train.edges())
    
print('Number of nodes of training set:', n)
print('Number of edges of training set:', m)

y_val = [1]*len(val_edges)

n_val_edges = len(val_edges)

# Create random pairs of nodes
for i in range(n_val_edges):
    n1 = nodes[randint(0, n-1)]
    n2 = nodes[randint(0, n-1)]
    (n1, n2) = (min(n1, n2), max(n1, n2))
    val_edges.append((n1, n2))
    
# Remove from val_edges edges that exist in both train and val

for edge in list(set(val_edges) & set(train_edges)):
    val_edges.remove(edge)
    
n_val_edges = len(val_edges) - len(y_val) #because we removed from val_edges edges that exist in both
y_val.extend([0]*n_val_edges)

Number of nodes of training set: 138499
Number of edges of training set: 982222


In [18]:
len(val_edges)

218715

In [16]:
adj = nx.adjacency_matrix(G_train) # Obtains the adjacency matrix of the training graph
adj = normalize_adjacency(adj) # Normalizes the adjacency matrix only by adding ones to diag
indices = np.array(adj.nonzero()) # Gets the positions of non zeros of adj into indices


/var/folders/pd/glcnl2497w5b6xn3p94tnwlr0000gn/T/ipykernel_44001/65278059.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G_train) # Obtains the adjacency matrix of the training graph


degs shape: (138499,)
inv_degs (138499,)
D_inv shape: (138499, 138499)
A_hat shape: (138499, 138499)


In [17]:
# features initializaed randomly because not yet ready
features_np = np.random.randn(G_train.number_of_nodes(), 8) # Generates node features randomly
features_np.shape

(138499, 8)

In [18]:
G_train.number_of_edges()*2 # = 1965036 

1964444

In [19]:
# Create class labels
y = np.zeros(4*G_train.number_of_edges())
y[:2*G_train.number_of_edges()] = 1 # Concatenated ones for edges indices and zeros for random indices.

# Transforms the numpy matrices/vectors to torch tensors.
features = torch.FloatTensor(features_np).to(device)
y = torch.LongTensor(y).to(device)
print(type(adj))
adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
print(type(adj))
indices = torch.LongTensor(indices).to(device)



<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'torch.Tensor'>


In [28]:
sparse_mx_to_torch_sparse_tensor(adj)

<class 'torch.Tensor'>


AttributeError: 'Tensor' object has no attribute 'tocoo'

In [24]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [31]:
# Hyperparameters
epochs = 20
n_hidden = 128
dropout_rate = 0.2
n_class = 2
n_features = features.shape[1]

# Creates the model and specifies the optimizer
model = GNN(n_features, n_hidden, n_class, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [33]:
import time

# Train model
model.train()
start_time = time.time()
for epoch in range(epochs):
    t = time.time()
    optimizer.zero_grad()
    rand_indices = torch.randint(0, features.size(0), (indices.size(0),indices.size(1)), device=adj.device)# We take random indices each time we run an epoch
    pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices.   
    output = model(features, adj, pairs) # we run the model that gives the output.
    loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
    print(type(loss_train), '\n', loss_train.shape)
    acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
    loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
    optimizer.step() # Performs a single optimization step (parameter update).
    
    if epoch % 5 == 0:
        print('Epoch: {:03d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'time: {:.4f}s'.format(time.time() - t),
             'total_time: {}min'.format(round((time.time() - start_time)/60)))

print("Optimization Finished in {} min!".format(round((time.time() - start_time)/60)))
print()

<class 'torch.Tensor'> 
 torch.Size([])
Epoch: 001 loss_train: 59.4502 acc_train: 0.4955 time: 15.8311s total_time: 0min
Optimization Finished in 0 min!



In [34]:
help(loss_train.backward)

Help on method backward in module torch._tensor:

backward(gradient=None, retain_graph=None, create_graph=False, inputs=None) method of torch.Tensor instance
    Computes the gradient of current tensor w.r.t. graph leaves.
    
    The graph is differentiated using the chain rule. If the tensor is
    non-scalar (i.e. its data has more than one element) and requires
    gradient, the function additionally requires specifying ``gradient``.
    It should be a tensor of matching type and location, that contains
    the gradient of the differentiated function w.r.t. ``self``.
    
    This function accumulates gradients in the leaves - you might need to zero
    ``.grad`` attributes or set them to ``None`` before calling it.
    See :ref:`Default gradient layouts<default-grad-layouts>`
    for details on the memory layout of accumulated gradients.
    
    .. note::
    
        If you run any forward ops, create ``gradient``, and/or call ``backward``
        in a user-specified CUDA strea

In [ ]:
node_pairs = np.array(np.transpose(val_edges))
pairs = torch.LongTensor(node_pairs).to(device)
output = model(features, adj, pairs)
y_pred = torch.exp(output)
y_pred = y_pred.detach().cpu().numpy()

In [ ]:
y_pred[190000]

In [ ]:
print('Log loss:', log_loss(y_val, y_pred[:,1]))

In [ ]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
pairs = torch.LongTensor(node_pairs).to(device)
output = model(features, adj, pairs)
y_pred = torch.exp(output)
y_pred = y_pred.detach().cpu().numpy()

# Compute log loss
y_test = np.loadtxt('y_test.txt', delimiter=',')[:,1]
y_pred = y_pred[:,1]
y_pred[y_pred>0.9999] = 0.9999
y_pred[y_pred<0.0001] = 0.0001
print('Log loss:', log_loss(y_test, y_pred))

In [ ]:
#adj = normalize_adjacency(adj) # Normalizes the adjacency matrix
#indices = np.array(adj.nonzero())

# Do we create the adjencency matrix based on the Training G ? And then we need to create one for the val G ?

# Combine input and output to get indices matrix undirected for the undirected Graph.
# You need to experiment with different hyperparameters and number of layers.

In [ ]:
features_np.shape

In [ ]:
indices.size()

In [ ]:
# features, adj, pairs
rand_indices.size()

In [ ]:
print(m)
indices.shape

In [ ]:
# Create a non(?) directed graph
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
node_to_idx = dict()
for i, node in enumerate(nodes):
    node_to_idx[node] = i
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Hyperparameters
epochs = 1000
n_hidden = 256
dropout_rate = 0.2

n_class = 2
n_nodes = G.number_of_nodes()
adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix
indices = np.array(adj.nonzero())
adj = normalize_adjacency(adj) # Normalizes the adjacency matrix
features_np = np.random.randn(n_nodes, 32) # Generates node features

# Create class labels
y = np.zeros(4*m)
y[:2*m] = 1

# Transforms the numpy matrices/vectors to torch tensors
features = torch.FloatTensor(features_np).to(device)
y = torch.LongTensor(y).to(device)
adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
indices = torch.LongTensor(indices).to(device)

# Creates the model and specifies the optimizer
model = GNN(features.shape[1], n_hidden, n_class, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train model
model.train()
for epoch in range(epochs):
    t = time.time()
    optimizer.zero_grad()
    rand_indices = torch.randint(0, features.size(0), (indices.size(0),indices.size(1)), device=adj.device)
    pairs = torch.cat((indices, rand_indices), dim=1)
    output = model(features, adj, pairs)
    loss_train = F.nll_loss(output, y)
    acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())
    loss_train.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
        print('Epoch: {:03d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'time: {:.4f}s'.format(time.time() - t))

print("Optimization Finished!")
print()

# # Read test data. Each sample is a pair of nodes
# node_pairs = list()
# with open('test.txt', 'r') as f:
#     for line in f:
#         t = line.split(',')
#         node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

# # Testing
# model.eval()
# node_pairs = np.array(np.transpose(node_pairs))
# pairs = torch.LongTensor(node_pairs).to(device)
# output = model(features, adj, pairs)
# y_pred = torch.exp(output)
# y_pred = y_pred.detach().cpu().numpy()

# # Compute log loss
# y_test = np.loadtxt('y_test.txt', delimiter=',')[:,1]
# y_pred = y_pred[:,1]
# y_pred[y_pred>0.9999] = 0.9999
# y_pred[y_pred<0.0001] = 0.0001
# print('Log loss:', log_loss(y_test, y_pred))

In [ ]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
pairs = torch.LongTensor(node_pairs).to(device)
output = model(features, adj, pairs)
y_pred = torch.exp(output)
y_pred = y_pred.detach().cpu().numpy()

In [ ]:
import pandas as pd

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])

In [ ]:
# y_pred_df.head()

y_pred_array = np.column_stack((range(len(y_pred)), y_pred_true))
# y_pred_df = pd.DataFrame(y_pred)
df_pred = pd.DataFrame({range(len(y_pred)), y_pred_true}, columns={'id','predicted'})#, columns={'id', 'predicted'}).astype({'id':'int'})
df_pred.head()

In [ ]:
# pd.DataFrame(y_pred_array, columns={'id', 'predicted'}).astype({'id':'int'}).head()

# pd.DataFrame(y_pred_array, columns={'id', 'predicted'}).astype({'id':'int'}).to_csv(
# "submission.csv", header=True, index=False
# )

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"submission.csv", header=True, index=True, index_label='id'
)

In [ ]:
# Write predictions to a file
predictions = zip(range(len(y_pred)), y_pred)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in y_pred_array:
        csv_out.writerow(row) 

CNN with Sigmoid to predict if they connected based on the abstract.

A good way to aggregate the text is to calculate the average (mean)

Another approach is to use directly a GNN. Take the abstract, compute the embeddng of the words. Take the mean of the node.
Then you can take the features.

For thr GN, we have only the 

pairs is a tensor, contains a pair of nodes that contains all the positive samples and some of the negative samples. y: half of them are equal to one, and half of them are connected. rand_indices are random pairs that are considered as not connected.

As we have a non directed. We can take twice every edge (2*m instead of 2*m for y. Or we can take the edges only once.

One vector for the abstract using the word2vec embedding or any other similar approach.

Or we can directly use a CNN.We can take a CNN and feed pais of abstracts in the CNN, the CNN will produce one vector for the first abstract and one vector for the second. We can combine these two vectors.

Then we can use an MLP to produce a vector, and then we can concatenate the two vectors from CNN and MLP.

There is a pretrained word embedding (Google provided a pretrained embedding).

Embedding of each word. Then the CNN will provide one vector for the abstract.

Each abstract has a different number of words. the representation of the CNN will have a fixed size of the embedding vector.


